In [1]:
# -*- coding: utf-8 -*-


'''
builds input values into python files for DayCent using latin hypercube sampling



requires pip installation of:
scikit-optimize
numpy
'''

import os
import numpy as np
import time
np.random.seed(int(time.time()))
import matplotlib.pyplot as plt
from skopt.space import Space
from skopt.sampler import Sobol
from skopt.sampler import Lhs
from skopt.sampler import Halton
from skopt.sampler import Hammersly
from skopt.sampler import Grid
from scipy.spatial.distance import pdist


In [2]:
# set up the comment string:

comment_str = '# 0 clteff_1 (0.0, 3.0) , # 1 clteff_2 = clteff_1 + 1.0, # 2 clteff_3 = clteff_1 + 1.0, '\
    '# 3 clteff_4 = ctleff_1, '\
    '# 4 cult_nt_date (124,145), # 5 cult_b_date = cult_nt_date + 1, '\
    '# 6 crop_sybn2_date cult_nt_date + 1, # 7 crop_plant_date cult_nt_date +1'\
    '# 8 feramt_n4_1 0.0-20.0 float, # 9 feramt_n4_2 0.0-5.0 float, # 10 feramt_n4_3 0.0-5.0 float, '\
    '# 11 feramt_n4_date = cult_nt_date +1, '\
    '# 12 rmvstr 0.0-1.0 float, # 13 remwsd 0.0-1.0 float, # 14 harv_g_date 272-292 int'
comment_str

'# 0 clteff_1 (0.0, 3.0) , # 1 clteff_2 = clteff_1 + 1.0, # 2 clteff_3 = clteff_1 + 1.0, # 3 clteff_4 = ctleff_1, # 4 cult_nt_date (124,145), # 5 cult_b_date = cult_nt_date + 1, # 6 crop_sybn2_date cult_nt_date + 1, # 7 crop_plant_date cult_nt_date +1# 8 feramt_n4_1 0.0-20.0 float, # 9 feramt_n4_2 0.0-5.0 float, # 10 feramt_n4_3 0.0-5.0 float, # 11 feramt_n4_date = cult_nt_date +1, # 12 rmvstr 0.0-1.0 float, # 13 remwsd 0.0-1.0 float, # 14 harv_g_date 272-292 int'

In [3]:
# make sure matrices directory exists for output files
dir = "./matrices"
if not os.path.isdir(dir):
    print("creating {}".format(dir))
    os.mkdir(dir)
else:
    print("{} already exists".format(dir))

./matrices already exists


In [4]:
%%time
n_samples = [10, 100, 1000, 10000, 100000]
# n_samples = [10, 20]

# all the variable spaces, see inline comments. I make them negative when they're actually going to be replaced
# programmatically so it's easy to spot in the resulting python files
space = Space([(0.0, 3.0),    # 0 clteff_1
               (-0.3, -0.0),  # 1 clteff_2 = clteff_1 + 1.0
               (-0.3, -0.0),  # 2 clteff_3 = clteff_1 + 1.0
               (-0.3, -0.0),  # 3 clteff_4 = clteff_1
               (124,145),     # 4 cult_nt_date (124,145)
               (-145, -124),  # 5 cult_b_date = cult_nt_date +1
               (-145, -124),  # 6 crop_sybn2_date = cult_nt_date +1
               (-145, -124),  # 7 crop_plant_date = cult_nt_date + 1
               (0.0, 20.0),   # 8 feramt_n4_1 0.0-20.0 float
               (0.0, 5.0),    # 9 feramt_n4_2 0.0-5.0 float
               (0.0, 5.0),    # 10 feramt_n4_3 0.0-5.0 float
               (-133, -113),  # 11 feramt_n4_date = cult_nt_date +1
               (0.0, 1.0),    # 12 rmvstr 0.0-1.0 float
               (0.0, 1.0),    # 13 remwsd 0.0-1.0 float
               (272, 292)     # 14 harv_g_date 272, 292 int
              ]) 

lhs = Lhs(lhs_type="classic", criterion=None)
for samplesize in n_samples:
    x = lhs.generate(space.dimensions, samplesize)
    file = open("matrices/LHS_og_{}.py".format(samplesize),"w")
    str_x = repr(x)
    file.write("og_matrix = ")
    file.write(str_x)
    file.close()

CPU times: user 1.69 s, sys: 132 ms, total: 1.82 s
Wall time: 1.82 s


In [5]:
print("done generating OG input matrix files")

done generating OG input matrix files


In [6]:
%%time

import sys
import importlib
# now we have to read them, fix them, rewrite them for the n+1 situation
for sz in n_samples:
    globals().update(importlib.import_module('matrices.LHS_og_{}'.format(sz)).__dict__)
    #print(og_matrix)
    for bit in og_matrix:
        '''
        print('astgc={}, astgc_d={}, clteff_1={}, clteff_2={}, clteff_3={}, clteff_4={}, '\
              'cult_nt_date={}, cult_b_date={}, feramt_1={}, feramt_2={}, feramt_3={}, '\
              'fert_n7_date= {}, crop_c6_date={}, pltm_date={}, feramt_n10_1={}, feramt_n10_2={}, feramt_n10_3={}, '\
              'feramt_n10_date={}, rmvstr={}, remwsd={}, harv_g_date={}'
              .format(bit[0], bit[1], bit[2], bit[3], bit[4], bit[5], bit[6], bit[7], 
                      bit[8], bit[9], bit[10], bit[11], bit[12], bit[13], bit[14], bit[15], bit[16], bit[17], bit[18], bit[19], bit[20]))
        '''
    #print("\n\n")
    
    for i in range(0, len(og_matrix)):
        # print(og_matrix[i])
        og_matrix[i] = [
            # clteff_1, clteff_2 = clteff_1 + 1, clteff_3 = clteff_1 +1, clteff_4 = clteff_1
            og_matrix[i][0], og_matrix[i][0] + 1.0,  og_matrix[i][0] + 1.0, og_matrix[i][0],
            #4: cult_nt_date, cult_b_date = cult_nt_date + 1, crop_sybn2_date = cult_nt_date +1, crop_plant_date = cult_nt_date+1
            og_matrix[i][4], og_matrix[i][4] + 1 , og_matrix[i][4] + 1, og_matrix[i][4] + 1,
            #8: feramt_n4_1, 9: feramt_n4_2, 10:feramt_n4_3
            og_matrix[i][8], og_matrix[i][9], og_matrix[i][10], 
            #11: feramt_n4_date = cult_nt_date + 1
            og_matrix[i][4] + 1,
            # 12 rmvstr 0.0-1.0 float, 13 remwsd 0.0-1.0 float
            og_matrix[i][12], og_matrix[i][13],
            # 20 harv_g_date 272-292 int
            og_matrix[i][14]
           ]
        #print(og_matrix[i])
    # print("\n\n")
    
    file = open("matrices/LHS_{}.py".format(sz),"w")
    file.write(comment_str)
    file.write("\n\n")
    file.write("matrix = {}".format(og_matrix))
    #file.write(str(og_matrix)
    file.close()

CPU times: user 4.27 s, sys: 943 ms, total: 5.22 s
Wall time: 5.2 s
